# <b> <font face = "Space Mono"> Práctica 1 </font> </b> 

<font face = "Rubik"> 

Durante todo el módulo ustedes serán los analistas de datos de una compañía de venta de insumos tecnológicos al público. A lo largo de las prácticas se harán cargo de la información de la empresa y realizarán el proceso completo de captura, limpieza, análisis, diagnóstico, documentación, explotación y publicación de resultados.

1.  La Dirección de Ventas ha solicitado las siguientes tablas a Marketing con el fin de que sean integradas:

    *  La tabla de puntos de venta propios, un Excel frecuentemente utilizado para contactar a cada sucursal, actualizada en 2021. <br>
  
    *  La tabla de empleados, un Excel mantenido por el personal administrativo de RRHH. <br>
    *  La tabla de proveedores, un Excel mantenido por un analista de otra dirección que ya no esta en la empresa. <br>
    *  La tabla de clientes, alojada en el CRM de la empresa. <br>
    *  La tabla de productos, un Excel mantenido por otro analista. <br>
    *  Las tablas de ventas, gastos y compras, tres archivos CSV generados a partir del sistema transaccional de la empresa. <br>
2.  Es necesario realizar la captura de esos archivos e ingestarlos dentro de nuestra base de datos. <br>

## <b> <font face = "Space Mono"> Resolución </font> </b> #


### <b> <font face = "Space Mono"> Creamos el catálogo de funciones y procedimientos </font> </b> 


In [ ]:
SET GLOBAL log_bin_trust_function_creators = 1;
DROP FUNCTION IF EXISTS `UC_Words`;
DELIMITER $$
CREATE DEFINER=`root`@`localhost` FUNCTION `UC_Words`( str VARCHAR(255) ) RETURNS varchar(255) CHARSET utf8
BEGIN  
  DECLARE c CHAR(1);  
  DECLARE s VARCHAR(255);  
  DECLARE i INT DEFAULT 1;  
  DECLARE bool INT DEFAULT 1;  
  DECLARE punct CHAR(17) DEFAULT ' ()[]{},.-_!@;:?/';  
  SET s = LCASE( str );  
  WHILE i < LENGTH( str ) DO  
     BEGIN  
       SET c = SUBSTRING( s, i, 1 );  
       IF LOCATE( c, punct ) > 0 THEN  
        SET bool = 1;  
      ELSEIF bool=1 THEN  
        BEGIN  
          IF c >= 'a' AND c <= 'z' THEN  
             BEGIN  
               SET s = CONCAT(LEFT(s,i-1),UCASE(c),SUBSTRING(s,i+1));  
               SET bool = 0;  
             END;  
           ELSEIF c >= '0' AND c <= '9' THEN  
            SET bool = 0;  
          END IF;  
        END;  
      END IF;  
      SET i = i+1;  
    END;  
  END WHILE;  
  RETURN s;  
END$$
DELIMITER ;
DROP PROCEDURE IF EXISTS `Llenar_dimension_calendario`;
DELIMITER $$
CREATE DEFINER=`root`@`localhost` PROCEDURE `Llenar_dimension_calendario`(IN `startdate` DATE, IN `stopdate` DATE)
BEGIN
    DECLARE currentdate DATE;
    SET currentdate = startdate;
    WHILE currentdate < stopdate DO
        INSERT INTO calendario VALUES (
                        YEAR(currentdate)*10000+MONTH(currentdate)*100 + DAY(currentdate),
                        currentdate,
                        YEAR(currentdate),
                        MONTH(currentdate),
                        DAY(currentdate),
                        QUARTER(currentdate),
                        WEEKOFYEAR(currentdate),
                        DATE_FORMAT(currentdate,'%W'),
                        DATE_FORMAT(currentdate,'%M'));
        SET currentdate = ADDDATE(currentdate,INTERVAL 1 DAY);
    END WHILE;
END$$
DELIMITER ;

### <b> <font face = "Space Mono"> Importamos las tablas </font> </b> 

*   <b> <font face = "Space Mono"> canal_venta </font> </b> 


In [ ]:
DROP TABLE IF EXISTS `canal_venta`;
CREATE TABLE IF NOT EXISTS `canal_venta` (
  `id_canal`			INTEGER,
  `canal` 				VARCHAR(50)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;
LOAD DATA INFILE '\CanalDeVenta.csv' 
INTO TABLE `canal_venta` 
FIELDS TERMINATED BY ',' ENCLOSED BY '\"' ESCAPED BY '\"' 
LINES TERMINATED BY '\n' IGNORE 1 LINES;

*   <b> <font face = "Space Mono"> cliente </font> </b> 


In [ ]:
DROP TABLE IF EXISTS cliente;
CREATE TABLE IF NOT EXISTS cliente (
	`id_cliente`					INTEGER,
	`provincia`			VARCHAR(50),
	`nombre_apellido`	VARCHAR(80),
	`domicilio`			VARCHAR(150),
	`telefono`			VARCHAR(30),
	`edad`				VARCHAR(5),
	`localidad`			VARCHAR(80),
	`x`				    VARCHAR(30),
	`y`				    VARCHAR(30),
	`col10`				VARCHAR(1)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;
LOAD DATA INFILE '\Clientes.csv'
INTO TABLE cliente
FIELDS TERMINATED BY ';' ENCLOSED BY '\"' ESCAPED BY '\"' 
LINES TERMINATED BY '\n' IGNORE 1 LINES;

*   <b> <font face = "Space Mono"> compra </font> </b> 


In [ ]:
DROP TABLE IF EXISTS `compra`;
CREATE TABLE IF NOT EXISTS `compra` (
  `id_compra`			INTEGER,
  `fecha` 				DATE,
  `id_producto`		INTEGER,
  `cantidad`			INTEGER,
  `precio`				DECIMAL(10,2),
  `id_proveedor`	INTEGER
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;
LOAD DATA INFILE '\Compra.csv' 
INTO TABLE `compra` 
FIELDS TERMINATED BY ',' ENCLOSED BY '' ESCAPED BY '' 
LINES TERMINATED BY '\n' IGNORE 1 LINES;

*   <b> <font face = "Space Mono"> empleado </font> </b> 


In [ ]:
DROP TABLE IF EXISTS empleado;
CREATE TABLE IF NOT EXISTS empleado (
	`id_empleado`					INTEGER,
	`apellido`					    VARCHAR(100),
	`nombre`						VARCHAR(100),
	`sucursal`					    VARCHAR(50),
	`sector`						VARCHAR(50),
	`cargo`						    VARCHAR(50),
	`salario`					    VARCHAR(30)
-- 	`salario`      					DECIMAL(10,2)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;
LOAD DATA INFILE '\Empleados.csv' 
INTO TABLE `empleado` 
FIELDS TERMINATED BY ',' ENCLOSED BY '\"' ESCAPED BY '\"' 
LINES TERMINATED BY '\n' IGNORE 1 LINES;

*   <b> <font face = "Space Mono"> gasto </font> </b> 


In [ ]:
DROP TABLE IF EXISTS `gasto`;
CREATE TABLE IF NOT EXISTS `gasto` (
  	`id_gasto` 		INTEGER,
  	`id_sucursal` 	INTEGER,
  	`id_tipo_gasto` INTEGER,
    `fecha`			DATE,
  	`monto` 		DECIMAL(10,2)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;	
LOAD DATA INFILE '\Gasto.csv' 
INTO TABLE `gasto` 
FIELDS TERMINATED BY ',' ENCLOSED BY '' ESCAPED BY '' 
LINES TERMINATED BY '\n' IGNORE 1 LINES;

*   <b> <font face = "Space Mono"> productos </font> </b> 


In [ ]:
DROP TABLE IF EXISTS producto;
CREATE TABLE IF NOT EXISTS producto (
	`id_producto`				INTEGER,
	`concepto`					VARCHAR(100),
	`tipo`						VARCHAR(50),
	`precio`					VARCHAR(30)
--  `precio`       				DECIMAL(10,2)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;
LOAD DATA INFILE '\Productos.csv' 
INTO TABLE `producto` 
FIELDS TERMINATED BY ',' ENCLOSED BY '\"' ESCAPED BY '\"' 
LINES TERMINATED BY '\n' IGNORE 1 LINES;

*   <b> <font face = "Space Mono"> proveedor </font> </b> 


In [ ]:
DROP TABLE IF EXISTS proveedor;
CREATE TABLE IF NOT EXISTS proveedor (
	`id_proveedor`	INTEGER,
	`nombre`		VARCHAR(80),
	`domicilio`		VARCHAR(150),
	`ciudad`		VARCHAR(80),
	`provincia`		VARCHAR(50),
	`pais`			VARCHAR(20),
	`departamento`	VARCHAR(80)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;
LOAD DATA INFILE '\Proveedores.csv' 
INTO TABLE proveedor
FIELDS TERMINATED BY ',' ENCLOSED BY '\"' ESCAPED BY '\"' 
LINES TERMINATED BY '\n' IGNORE 1 LINES;

*   <b> <font face = "Space Mono"> sucursales </font> </b> 


In [ ]:
DROP TABLE IF EXISTS sucursal;
CREATE TABLE IF NOT EXISTS sucursal (
	`id_sucursal`			INTEGER,
	`sucursal`	            VARCHAR(40),
	`domicilio`	            VARCHAR(150),
	`localidad`	            VARCHAR(80),
	`provincia`	            VARCHAR(50),
	`latitud`	            VARCHAR(30),
	`longitud`	            VARCHAR(30)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;
LOAD DATA INFILE '\Sucursales.csv' 
INTO TABLE sucursal
FIELDS TERMINATED BY ';' ENCLOSED BY '\"' ESCAPED BY '\"' 
LINES TERMINATED BY '\n' IGNORE 1 LINES;

*   <b> <font face = "Space Mono"> tipo_gaste </font> </b> 


In [ ]:
DROP TABLE IF EXISTS `tipo_gasto`;
CREATE TABLE IF NOT EXISTS `tipo_gasto` (
  `id_tipo_gasto`       INT(11) NOT NULL AUTO_INCREMENT,
  `descripcion`         VARCHAR(100) NOT NULL,
  `monto_aproximado`    DECIMAL(10,2) NOT NULL,
  PRIMARY KEY (`id_tipo_gasto`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;
LOAD DATA INFILE '\TiposDeGasto.csv' 
INTO TABLE `tipo_gasto` 
FIELDS TERMINATED BY ',' ENCLOSED BY '\"' ESCAPED BY '\"' 
LINES TERMINATED BY '\n' IGNORE 1 LINES;

*   <b> <font face = "Space Mono"> venta </font> </b> 


In [ ]:
DROP TABLE IF EXISTS `venta`;
CREATE TABLE IF NOT EXISTS `venta` (
  `id_venta`				INTEGER,
  `fecha` 				DATE NOT NULL,
  `fecha_entrega` 		DATE NOT NULL,
  `id_canal`			INTEGER, 
  `id_cliente`			INTEGER, 
  `id_sucursal`			INTEGER,
  `id_empleado`			INTEGER,
  `id_producto`			INTEGER,
  `precio`				VARCHAR(30),
  `cantidad`			VARCHAR(30)
  -- `precio`			DECIMAL(10,2),
  -- `cantidad`			INTEGER
  -- eliminar: `precio`
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;
LOAD DATA INFILE '\Venta.csv' 
INTO TABLE `venta` 
FIELDS TERMINATED BY ',' ENCLOSED BY '' ESCAPED BY '' 
LINES TERMINATED BY '\n' IGNORE 1 LINES;

### <b> <font face = "Space Mono"> Generamos la dimensión calendario </font> </b> 

In [ ]:
DROP TABLE IF EXISTS `calendario`;
CREATE TABLE calendario (
        id_calendario                      INTEGER PRIMARY KEY,  -- year*10000+month*100+day
        fecha                 	DATE NOT NULL,
        anio                    INTEGER NOT NULL,
        mes                   	INTEGER NOT NULL, -- 1 to 12
        dia                     INTEGER NOT NULL, -- 1 to 31
        trimestre               INTEGER NOT NULL, -- 1 to 4
        semana                  INTEGER NOT NULL, -- 1 to 52/53
        dia_nombre              VARCHAR(9) NOT NULL, -- 'Monday', 'Tuesday'...
        mes_nombre              VARCHAR(9) NOT NULL -- 'January', 'February'...
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;

/*LOAD DATA INFILE 'C:\\Users\\HP\\Desktop\\Visual_Studio_Code\\DTS_01_Bootcamp\\DS_M3\\Clase_01\\Homework_Resuelto\\Calendario.csv' 
INTO TABLE calendario
FIELDS TERMINATED BY ',' ENCLOSED BY '' ESCAPED BY '' 
LINES TERMINATED BY '\n' IGNORE 1 LINES;*/

/*TRUNCATE TABLE calendario;*/
CALL Llenar_dimension_calendario('2015-01-01','2020-12-31');
SELECT * FROM calendario;